# PyOR Quantum
## Author: Vineeth Thalakottoor
## Introduction to Quadrupole Hamiltonians

In [1]:
# Define the source path
SourcePath = '/media/HD2/Vineeth/PostDoc_Simulations/Github/PyOR_V1/PyOR_Combined/Source_Doc'

# Add source path
import sys
sys.path.append(SourcePath)

import numpy as np
# Import PyOR package
from PyOR_QuantumSystem import QuantumSystem as QunS
from PyOR_Hamiltonian import Hamiltonian
from PyOR_QuantumLibrary import QuantumLibrary
import PyOR_SphericalTensors as ST
import PyOR_Rotation as Rot
QLib = QuantumLibrary()

In [2]:
# Define the spin system
Spin_list = {"A" : "H2"}
QS = QunS(Spin_list,PrintDefault=False)

# initialize the system
QS.Initialize()

### Set parameters

In [3]:
# Master Equation
QS.PropagationSpace = "Hilbert"
QS.MasterEquation = "Redfield"

# B0 Field in Tesla, Static Magnetic field (B0) along Z
QS.B0 = 9.4

# Offset Frequency in rotating frame (Hz)
QS.OFFSET["A"] = 0.0

# Define initial and final Spin Temperature
QS.I_spintemp["A"] = 300.0
QS.F_spintemp["A"] = 300.0


# Relaxation Process
QS.Rprocess = "Phenomenological"
QS.R1 = 1
QS.R2 = 2

QS.Update()

Rotating frame frequencies: {'A': -386011000.0}
Offset frequencies: {'A': 0.0}
Initial spin temperatures: {'A': 300.0}
Final spin temperatures: {'A': 300.0}
Radiation damping gain: {'A': 0}
Radiation damping phase: {'A': 0}

Rprocess = Phenomenological
RelaxParDipole_tau = 0.0
DipolePairs = []
RelaxParDipole_bIS = []


### Zeeman Hamiltonians

In [4]:
# generate Larmor Frequencies
QS.print_Larmor = True
Ham = Hamiltonian(QS)

Larmor Frequency in MHz:  [-61.43555874]


In [5]:
# Lab Frame Hamiltonian
Hz_lab = Ham.Zeeman()
Hz_lab.Inverse2PI().matrix

Matrix([
[-61435560.0110852, 0,                0],
[                0, 0,                0],
[                0, 0, 61435560.0110852]])

In [6]:
# Rotating Frame Hamiltonian
Hz = Ham.Zeeman_RotFrame()
Hz.Inverse2PI().matrix

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

### Quadrupole tensor PAF

In [7]:
QuadrupoleCoupling = 100 # Hz
eta = 0.2

A = "A"

IT_PAF = Ham.InteractionTensor_PAF_Quadrupole(A,QuadrupoleCoupling,eta)
IT_PAF.Inverse2PI().matrix

Matrix([
[-30.0,     0,    0],
[    0, -20.0,    0],
[    0,     0, 50.0]])

In [8]:
PAF_Decom = Ham.InteractionTensor_PAF_Decomposition(IT_PAF)
PAF_Decom

{'Isotropic': 0.0, 'Anisotropy': 50.0, 'Asymmetry': 0.19999999999999998}

### Quadrupole tensor LAB

In [9]:
phi,theta = 20.0,40.0

IT_LAB = Ham.InteractionTensor_LAB(IT_PAF,phi,theta,0)
IT_LAB.Round(3).matrix

Matrix([
[  2.245,   46.555, 232.583],
[ 46.555, -108.719,  84.653],
[232.583,   84.653, 106.474]])

In [10]:
LAB_Decom = Ham.InteractionTensor_LAB_Decomposition(IT_LAB)
LAB_Decom['Isotropic'].matrix

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

In [11]:
LAB_Decom["Symmetric"].Inverse2PI().Round(3).matrix

Matrix([
[ 0.357,   7.409, 37.017],
[ 7.409, -17.303, 13.473],
[37.017,  13.473, 16.946]])

In [12]:
LAB_Decom["Antisymmetric"].Inverse2PI().Round(3).matrix

Matrix([
[0, 0, 0],
[0, 0, 0],
[0, 0, 0]])

### Quadrupole Hamiltonian in Lab frame (Secular)

In [13]:
Hdd_LAB_Truncates = Ham.Interaction_Hamiltonian_LAB_Quadrupole_Secular(A, QuadrupoleCoupling, eta,theta,phi)
Hdd_LAB_Truncates.Inverse2PI().Round(3).matrix

Matrix([
[8.715,       0,     0],
[    0, -17.429,     0],
[    0,       0, 8.715]])

### Quadrupole Hamiltonian in Lab frame (Cartesian, Euler rotation)

In [14]:
Hdd_LAB_Euler = Ham.Interaction_Hamiltonian_Catesian_Euler(A,IT_PAF,A,0,theta,phi)
Hdd_LAB_Euler.Inverse2PI().Round(3).matrix

Matrix([
[           8.715,  27.447 - 1.461*I,  12.455 + 2.462*I],
[27.447 + 1.461*I,           -17.429, -27.447 + 1.461*I],
[12.455 - 2.462*I, -27.447 - 1.461*I,             8.715]])

### Quadrupole Hamiltonian in Lab frame (Cartesian, Wigner rotation)

In [15]:
Hdd_LAB_Wig = Ham.Interaction_Hamiltonian_Catesian_Wigner(A,IT_PAF,A,0,theta,phi)
Hdd_LAB_Wig.Inverse2PI().Round(3).matrix

Matrix([
[           8.715,  27.447 - 1.461*I,  12.455 + 2.462*I],
[27.447 + 1.461*I,           -17.429, -27.447 + 1.461*I],
[12.455 - 2.462*I, -27.447 - 1.461*I,             8.715]])

### Quadrupole Hamiltonian in Lab frame (Spherical Tensors, Wigner Rotation)

In [16]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,A,"spin-spin","all", 0,theta,phi)
Hdd_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[           8.715,  27.447 + 1.461*I,  12.455 - 2.462*I],
[27.447 - 1.461*I,           -17.429, -27.447 - 1.461*I],
[12.455 + 2.462*I, -27.447 + 1.461*I,             8.715]])

In [17]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,A,"spin-spin","secular", 0,theta,phi)
Hdd_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[8.715,       0,     0],
[    0, -17.429,     0],
[    0,       0, 8.715]])

In [18]:
Hdd_LAB_ST = Ham.Interaction_Hamiltonian_SphericalTensor(A,IT_PAF,A,"spin-spin","secular + pseudosecular", 0,theta,phi)
Hdd_LAB_ST.Inverse2PI().Round(3).matrix

Matrix([
[8.715,       0,     0],
[    0, -17.429,     0],
[    0,       0, 8.715]])